# 環境設置

## 載入套件
- selenium、pandas、requests、bs4、time

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  # 設定 driver 的行為
from selenium.webdriver.support.ui import Select  # 選擇＂下拉式選單＂
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException, TimeoutException  # 載入常見錯誤
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities  # 更改載入策略
from selenium.webdriver.support.ui import WebDriverWait  # 等待機制
from selenium.webdriver.support import expected_conditions as EC  # 預期事件
from selenium.webdriver.common.by import By  # 找尋元素的方法
import pandas as pd  # 載入 pandas
import pandas.io.formats.excel  # 輸出自定義格式 Excel
import requests
import requests.packages.urllib3

requests.packages.urllib3.disable_warnings()  # 關閉錯誤警告
from urllib.request import HTTPError  # 載入 HTTPError
from bs4 import BeautifulSoup
import time  # 強制等待

## 設定 driver 的參數：options、desired_capabilities

In [2]:
my_options = Options()
my_options.add_argument('--incognito')  # 開啟無痕模式
# my_options.add_argument('--start-maximized')  # 視窗最大化
# my_options.add_argument('--headless')  # 不開啟實體瀏覽器
my_capabilities = DesiredCapabilities.CHROME
my_capabilities['pageLoadStrategy'] = 'eager'  # 頁面加載策略：HTML 解析成 DOM

## organize_columns(df1)
- 處理欄位：圖書館、館藏地（c2）、索書號（c3）、館藏狀態（c4）、連結
- 如果 df1 是裝著 DataFrame 的 list，則就合併它們；否則（df1 是 DataFrame），就接著執行以下敘述。
- 丟掉垃圾欄位，整理成要呈現的表格
- 新增必要欄位（圖書館、連結）
- 填滿 NaN（用 ffill 的 方式）

# 自定義函式

In [118]:
def organize_columns(df1):
    # 合併全部的 DataFrame
    try:
        df1 = pd.concat(df1, axis=0, ignore_index=True)
    except:
        df1.reset_index(drop=True, inplace=True)

    # 處理 column 2：館藏地
    c2 = [
        '分館/專室', '館藏地/室', '館藏室', '館藏地/館藏室', '館藏地', '典藏館', '館藏位置', '館藏地/區域',
        '典藏地名稱', '館藏地/館別', '館藏地(已外借/總數)'
    ]
    df1['c2'] = ''
    for c in c2:
        try:
            df1['c2'] += df1[c]
        except:
            pass

    # 處理 column 3：索書號
    c3 = ['索書號', '索書號/期刊合訂本卷期', '索書號 / 部冊號']
    df1['c3'] = ''
    for c in c3:
        try:
            df1['c3'] += df1[c]
        except:
            pass

    # 處理 column 4：館藏狀態
    c4 = [
        '館藏位置(到期日期僅為期限，不代表上架日期)', '狀態/到期日', '目前狀態 / 到期日', '館藏狀態', '處理狀態',
        '狀態 (說明)', '館藏現況 說明', '目前狀態/預計歸還日期', '圖書狀況 / 到期日', '調閱說明', '借閱狀態',
        '狀態', '館藏狀態(月-日-西元年)', '圖書狀況', '現況/異動日', 'Unnamed: 24'
    ]
    df1['c4'] = ''
    for c in c4:
        try:
            df1['c4'] += df1[c]
        except:
            pass

    # 直接生成另一個 DataFrame
    df2 = pd.DataFrame()
    df2['圖書館'] = df1['圖書館']
    df2['館藏地'] = df1['c2']
    df2['索書號'] = df1['c3']
    df2['館藏狀態'] = df1['c4']
    df2['連結'] = df1['連結']

    # 遇到值為 NaN時，將前一列的值填補進來
    df2.fillna(method="ffill", axis=0, inplace=True)

    return df2

## set_excel(df, directory)
- 待

In [4]:
def set_excel(df, directory):
    # B｢圖書館｣、C「館藏地」、D「索書號」、E「館藏狀態」、F「連結」
    pandas.io.formats.excel.header_style = None  # 標題格式清除
    writer = pd.ExcelWriter(directory)
    df.to_excel(writer, sheet_name="搜尋結果")

    workbook1 = writer.book
    worksheets = writer.sheets
    worksheet1 = worksheets["搜尋結果"]

    # 測試
    cell_format = workbook1.add_format({
        "font_name": "微軟正黑體",
        "font_size": 16,
        "align": "left",
        #         "border": 80,
    })
    worksheet1.set_column("B:F", 40, cell_format)

    # 設定單元格的寬度
    #     worksheet1.set_column("B:F", 35)

    writer.save()
    print("爬取完成")

## wait_for_element_present(element_position, waiting_time=5, by=By.CSS_SELECTOR)
- 用法：
    - 等待 element 出現，每間隔 0.5 秒定位一次，直到 5 秒。如果定位 element 成功，回傳 element；否則，回傳 None。
- 參數：
    - element_position：元素位置，預設 CSS selector
    - waiting_time：等待時間，預設 5 秒
    - by：定位方式，預設 By.CSS_SELECTOR

In [5]:
def wait_for_element_present(element_position, waiting_time=5, by=By.CSS_SELECTOR):
    try:
        element = WebDriverWait(driver, waiting_time).until(
            EC.presence_of_element_located((by, element_position)))
    except:
        return
    else:
        return element

## wait_for_element_clickable(element_position, waiting_time=5, by=By.PARTIAL_LINK_TEXT)
- 同上

In [6]:
def wait_for_element_clickable(element_position, waiting_time=5, by=By.PARTIAL_LINK_TEXT):
    try:
        element = WebDriverWait(driver, waiting_time).until(
            EC.element_to_be_clickable((by, element_position)))
    except:
        return
    else:
        return element

## wait_for_url_changed(old_url, waiting_time=10)
- 用法：
    - 等待網址改變（輸入的網址和現在的網址進行比較），每間隔 0.5 秒檢查一次，直到 10 秒。如果網址有改變，回傳 True；否則，回傳 None。
- 參數：
    - old_url：舊網址
    - waiting_time：等待時間，預設 10 秒

In [7]:
def wait_for_url_changed(old_url, waiting_time=10):
    try:
        WebDriverWait(driver, time).until(EC.url_changes(old_url))
    except:
        return
    else:
        return True

## accurately_find_table_and_read_it(table_position, table_position2=0)
- 用法：
    - 精準定位 table 並讀取成 pd.DataFrame。如果定位 table 成功，回傳 table；否則，回傳 None。
    - 為 table 增加＂圖書館＂和＂連結＂的欄位。
- 參數：
    - table_position：table 位置，預設 CSS selector

In [8]:
def accurately_find_table_and_read_it(table_position, table_position2=0):
    try:
        if not wait_for_element_present(table_position):
            print(f'找不到 {table_position}！')
            return
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table_innerHTML = soup.select(table_position)[table_position2]
        tgt = pd.read_html(str(table_innerHTML), encoding='utf-8')[0]
        # tgt['圖書館'], tgt['連結'] = org, driver.current_url
    except:
        return
    else:
        return tgt

## select_ISBN_strategy(select_position, option_position, waiting_time=30)
- 用法：
    - 等待 select 出現，並選擇以 ISBN 方式搜尋
- 參數：
    - select_position：select 位置，預設 name
    - option_position：option 位置，預設 value
    - waiting_time：等待時間，預設 30 秒

In [9]:
def select_ISBN_strategy(select_position, option_position, waiting_time=30):
    time.sleep(0.5)
    search_field = WebDriverWait(driver, waiting_time).until(EC.presence_of_element_located((By.NAME, select_position)))
    select = Select(search_field)
    select.select_by_value(option_position)

## search_ISBN(ISBN, input_position, waiting_time=10)
- 用法：
    - 等待 input 出現，輸入 ISBN 並按下 ENTER
- 參數：
    - ISBN：ISBN
    - input_position：input 位置，預設 name
    - waiting_time：等待時間，預設 10 秒

In [10]:
def search_ISBN(ISBN, input_position, waiting_time=10):
    time.sleep(0.5)
    search_input = WebDriverWait(driver, waiting_time).until(EC.presence_of_element_located((By.NAME, input_position)))
    search_input.send_keys(ISBN)
    search_input.send_keys(Keys.ENTER)

# 爬蟲程式

## 宜蘭縣公共圖書館(org, org_url, ISBN)
- 最後更新：7/27
- 『函式完成度』：極高

### 函式說明

- 『運作的原理』：
- 『適用的機構』：[宜蘭縣公共圖書館](https://webpac.ilccb.gov.tw/advanceSearch)、[桃園市立圖書館](https://webpac.typl.gov.tw/advanceSearch)、[高雄市立圖書館](https://webpacx.ksml.edu.tw/advanceSearch)、[屏東縣公共圖書館](https://library.pthg.gov.tw/advanceSearch)、[花蓮縣公共圖書館](https://center.hccc.gov.tw/advanceSearch)、[澎湖縣圖書館](https://webpac.phlib.nat.gov.tw/advanceSearch)、[國立雲林科技大學](https://www.libwebpac.yuntech.edu.tw/advanceSearch)、[國家電影及視聽文化中心](https://lib.tfi.org.tw/advanceSearch)
- 『能處理狀況』：判斷搜尋結果有沒有超過一筆、只有一筆搜尋結果有沒有跳轉、[多筆](https://webpac.typl.gov.tw/search?searchField=ISBN&searchInput=986729193X)、找不到書、[不斷的點擊＂載入更多＂](https://webpac.ilccb.gov.tw/bookDetail/419482?qs=%7B%5Eurl3%2C%2Fsearch4%2Cquery%5E%3A%7B%5Ephonetic3%2C04%2CqueryType3%2C04%2C%2Cs23%2CISBN4%2C%2Cs13%2C9789573317241%5E%7D%7D)
- 『下一步優化』：

### 函式本體

In [15]:
def click_more_btn(driver):
    try:
        while True:
            more_btn = wait_for_element_present('載入更多', by=By.PARTIAL_LINK_TEXT)
            if not more_btn:
                return
            more_btn.click()
            time.sleep(2)  # 不得已的強制等待
    except:
        return

In [16]:
def 宜蘭縣公共圖書館(org, org_url, ISBN):
    try:
        table = []

        driver.get(org_url)
        select_ISBN_strategy('searchField', 'ISBN')
        search_ISBN(ISBN, 'searchInput')

        # 一筆
        if wait_for_element_present('.bookplace_list > table', 10):
            click_more_btn(driver)
            tgt = accurately_find_table_and_read_it('.bookplace_list > table')
            tgt['圖書館'], tgt['連結'] = org, driver.current_url
            table.append(tgt)
        # 多筆
        elif wait_for_element_present('.data_all .data_quantity2 em', 5):
            tgt_urls = []
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            anchors = soup.select('.bookdata > h2 > a')
            for anchor in anchors:
                tgt_urls.append(org_url.replace('advanceSearch', '') + anchor['href'])
            for tgt_url in tgt_urls:
                driver.get(tgt_url)
                if wait_for_element_present('.bookplace_list > table', 10):
                    click_more_btn(driver)
                    tgt = accurately_find_table_and_read_it('.bookplace_list > table')
                    tgt['圖書館'], tgt['連結'] = org, driver.current_url
                    table.append(tgt)
        # 無
        else:
            print(f'在「{org}」找不到「{ISBN}」')
            return

        table = organize_columns(table)
    except Exception as e:
        print(e)
        return
    else:
        return table

### 函式測試

In [ ]:
if __name__ == '__main__':
    driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)
    table = 宜蘭縣公共圖書館(
        org='宜蘭縣公共圖書館',
        org_url='https://webpac.ilccb.gov.tw/advanceSearch',
        ISBN='9789869109321'
    )
table

## webpac_jsp_crawler(org, org_url, ISBN)
- 『函式完成度』：極高

### 函式說明
- 『運作的原理』：
    - 使用 selenium 進行搜索。
    - 大量使用 wait 機制，來應對加載過慢的網頁（例：[佛光大學](http://libils.fgu.edu.tw/webpacIndex.jsp)）
    - 當搜尋結果只有一筆時，有些網站會直接進入＂詳細書目＂（例：[國立宜蘭大學](https://lib.niu.edu.tw/webpacIndex.jsp)）
        - 還是會停留在＂搜尋結果＂頁面，但大部分會看不到，網址仍會改變，所以無法用網址判定
    - 當搜尋結果有多筆時，會要切換到 iframe 爬取。
    - 有些＂詳細書目＂會有沒有表格的情況（例：[中華科大](http://192.192.231.232/bookDetail.do?id=260965&nowid=3&resid=188809854)）
- 『適用的機構』：國立宜蘭大學、佛光大學、嘉南藥理大學、……
- 『能處理狀況』：[一筆](http://webpac.meiho.edu.tw/bookDetail.do?id=194508)、[無](http://webpac.meiho.edu.tw/bookSearchList.do?searchtype=simplesearch&search_field=ISBN&search_input=97895733172411&searchsymbol=hyLibCore.webpac.search.common_symbol&execodehidden=true&execode=&ebook=)、[多筆](http://webpac.meiho.edu.tw/bookSearchList.do?searchtype=simplesearch&execodeHidden=true&execode=&search_field=ISBN&search_input=9789573317241&searchsymbol=hyLibCore.webpac.search.common_symbol&resid=189006169&nowpage=1#searchtype=simplesearch&execodeHidden=true&execode=&search_field=ISBN&search_input=9789573317241&searchsymbol=hyLibCore.webpac.search.common_symbol&resid=189006169&nowpage=1)、[無表格](http://192.192.231.232/bookDetail.do?id=260965&nowid=3&resid=188809854)
- 『下一步優化』：
    - 統一 search_input.submit() 和 search_input.send_keys(Keys.ENTER)？

### 函式本體

In [13]:
def webpac_jsp_crawler(org, org_url, ISBN):
    try:
        table = []
        
        driver.get(org_url)
        select_ISBN_strategy('search_field', 'ISBN')  # select.select_by_value('STANDARDNO') -> 北科
        search_ISBN(ISBN, 'search_input')
        
        # 搜尋結果的數量為＂一筆＂
        if wait_for_element_present('div.mainCon'):
            if not wait_for_element_present('table.order'):
                return
            tgt = accurately_find_table_and_read_it('table.order')
            table.append(tgt)
        # 搜尋結果的數量為＂多筆＂和＂零筆＂
        elif wait_for_element_present('iframe#leftFrame'):
            iframe = driver.find_element_by_id('leftFrame')
            driver.switch_to.frame(iframe)
            # 切換到 <frame> 需要時間，否則會無法讀取
            time.sleep(1)
            # 解析 html，以取得 tgt_urls
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            # 判斷是不是＂零筆＂
            if soup.find('em', {'id': 'totalpage'}).text == '0':
                print(f'您所輸入的 ISBN 碼：{ISBN}，共查得 0 筆。')
                return
            anchors = soup.find_all('a', 'bookname')
            # tgt_urls 為各個＂詳細書目＂的網址
            tgt_urls = []
            for anchor in anchors:
                tgt_urls.append(org_url.replace('webpacIndex.jsp', '') + anchor['href'])
            # 取得 tgt_urls 後，開始進入 tgt_url
            for tgt_url in tgt_urls:
                # 進入＂詳細書目＂
                driver.get(tgt_url)
                # 等待元素出現，如果出現，那麼抓取 DataFrame；如果沒出現，那麼跳出迴圈
                if not wait_for_element_present('table.order'):
                    continue  # 暫停＂本次＂迴圈，以下敘述不會執行
                tgt = accurately_find_table_and_read_it('div.allDetail table.order')
                table.append(tgt)
        table = organize_columns(table)
    except Exception as e:
        print(e)
        return
    else:
        return table

### 函式測試

In [14]:
if __name__ == '__main__':
    driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)
    table = webpac_jsp_crawler(
        org='美和科技大學',
        org_url='http://192.192.231.232/webpacIndex.jsp',
        ISBN='9573317249'
    )
table

'圖書館'


## easy_crawler(table_position, org, org_url, ISBN)
- 『函式完成度』：高

### 函式說明
- 『運作的原理』：直接進入＂詳細書目＂頁面後，使用 pd.read_html 抓取 table，但每個機構的 table 位置不同，需要在 function 將此當作一個 parameter。
- 『適用的機構』：國立臺灣海洋大學、國立臺灣師範大學、中央研究院、國立陽明大學、國立中央大學、中原大學、國立臺灣科技大學、中國文化大學、輔仁大學
- 『能處理狀況』：HTTPError
- 『下一步優化』：師大查詢結果有多本（https://opac.lib.ntnu.edu.tw/search*cht/i?SEARCH=9789573229131 ）

### 函式本體

In [ ]:
def easy_crawler(table_position, org, org_url, ISBN):
    try:
        # 組合成書本的網址
        tgt_url = org_url + ISBN
        # 載入 html，如果發生 HTTPError，那麼就使用 requests.get(url, verify=False)
        try:
            tgt = pd.read_html(tgt_url, encoding="utf-8")
        except HTTPError:
            resp = requests.get(tgt_url,
                                verify=False)  # 設定 verify=False，以解決 SSLError
            tgt = pd.read_html(resp.text, encoding="utf-8")
        # 定位表格
        table = tgt[table_position]
        table['圖書館'], table['連結'] = org, tgt_url
        table = organize_columns(table)
        return table  # 完成抓取 table
    except:
        print(f'《{ISBN}》在「{url}」無法爬取')

### 函式測試

In [ ]:
if __name__ == '__main__':
    table = easy_crawler(
        table_position=4,
        org='國立臺灣師範大學',
        org_url='https://opac.lib.ntnu.edu.tw/search*cht/i?SEARCH=',
        ISBN='9789869109321'
    )
table

## 國立臺灣師範大學(org, org_url, ISBN)
- 最後編輯：2021/07/24
- 『函式完成度』：極高

### 函式說明
- 『運作的原理』：待輸入
- 『適用的機構』：[國立臺灣師範大學](https://opac.lib.ntnu.edu.tw/screens/opacmenu_chi.html)、[國立臺灣科技大學](https://sierra.lib.ntust.edu.tw/search~S1*cht)、[國立臺灣海洋大學](https://ocean.ntou.edu.tw/*cht)、[中原大學](http://cylis.lib.cycu.edu.tw/search*cht)、[逢甲大學](https://innopac.lib.fcu.edu.tw/search*cht)、[朝陽科技大學](https://millennium.lib.cyut.edu.tw/search*cht)、[國立中山大學](https://dec.lib.nsysu.edu.tw/search*cht)、[國立高雄師範大學](https://nknulib.nknu.edu.tw/search*cht)、[文藻外語大學](https://libpac.wzu.edu.tw/search*cht)、[大仁科技大學](http://lib.tajen.edu.tw/search*cht)
- 『能處理狀況』：一筆、無
- 『下一步優化』：
    - 待輸入
    - 待輸入

In [ ]:
def 國立臺灣師範大學(org, org_url, ISBN):
    driver.get(org_url)
    
    if not wait_for_element_clickable('ISBN', by=By.PARTIAL_LINK_TEXT):
        print('定位不到 ISBN（PARTIAL_LINK_TEXT）')
        return
    driver.find_element(By.PARTIAL_LINK_TEXT, 'ISBN').click()
    
    search_ISBN(ISBN, 'SEARCH')
    
    if not wait_for_element_present('table.bibItems'):
        print(f'在「{org}」找不到「{ISBN}」')
        return
    
    table = accurately_find_table_and_read_it('table.bibItems')
    table['圖書館'], table['連結'] = org, driver.current_url
    table = organize_columns(table)
    return table

In [ ]:
if __name__ == '__main__':
    driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)
    table = 國立臺灣師範大學(
        org='逢甲大學',
        org_url='https://innopac.lib.fcu.edu.tw/search*cht',
        ISBN='9789868879348'  # 國家為什麼會失敗
    )

## 中央研究院(org, org_url, ISBN)
- 最後編輯：2021/07/24
- 『函式完成度』：極高

### 函式說明
- 『運作的原理』：待輸入
- 『適用的機構』：[中央研究院](https://las.sinica.edu.tw/*cht)、[中國文化大學](https://webpac.pccu.edu.tw/*cht)、[輔仁大學](https://library.lib.fju.edu.tw:444/)、[國立陽明交通大學](https://library.ym.edu.tw/screens/opacmenu_cht_s7.html)、國立中央大學
- 『能處理狀況』：一筆、無
- 『下一步優化』：
    - 待輸入
    - 待輸入

In [ ]:
def 中央研究院(org, org_url, ISBN):
    driver.get(org_url)
    
    select_ISBN_strategy('searchtype', 'i')
    
    search_ISBN(ISBN, 'searcharg')

    if not wait_for_element_present('table.bibItems'):
        print(f'在「{org}」找不到「{ISBN}」')
        return

    table = accurately_find_table_and_read_it('table.bibItems')
    table['圖書館'], table['連結'] = org, driver.current_url
    table = organize_columns(table)
    return table

In [ ]:
driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)

In [ ]:
中央研究院('國立臺灣師範大學', 'https://www.lib.ntnu.edu.tw/mobile/holding/index.jsp', '9789869109321')

## webpac_ajax_page_crawler(org, org_url, ISBN)
- 『函式完成度』：極高

### 函式說明
- 『運作的原理』：使用 selenium 進行搜索，進入＂詳細書目＂頁面後，從該網址分析並得到 mid，在由此進入 ajax_page。
- 『適用的機構』：新北市立圖書館、高雄市立空中大學、國立屏東大學
- 『能處理狀況』：判斷搜尋結果有沒有超過一筆、只有一筆搜尋結果有沒有跳轉、找不到書
- 『下一步優化』：當搜尋無結果時，可以直接結束。

### 函式本體

In [ ]:
def webpac_ajax_page_crawler(org, org_url, ISBN):
    try:
        # 進入＂搜尋主頁＂
        driver.get(org_url)
        # 等待點擊＂進階查詢＂按鈕，接著點擊
        WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.LINK_TEXT, '進階查詢'))).click()
        # 等待定位＂下拉式選單＂，選擇以 ISBN 方式搜尋
        search_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'as_type_1')))
        select = Select(search_field)
        select.select_by_value('i')
        # 定位＂搜尋欄＂，輸入 ISBN
        search_input = driver.find_element_by_id('as_keyword_1')
        search_input.send_keys(ISBN)
        search_input.send_keys(
            Keys.ENTER)  # 無法 submit()，用 send_keys(keys.ENTER) 來替代

        # 在＂搜尋結果頁面＂，等待定位＂詳細書目＂。
        # try-except 來判斷有沒有在＂搜尋結果頁面＂
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.LINK_TEXT, '詳細書目')))
        except:
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, 'div.book-detail')))

                # 抓取方式：找出 mid 後，進入 ajax pag 抓取 DataFrame
                org_url = org_url.replace('/search.cfm', '')
                tgts = []
                url = driver.current_url
                mid = url.split('mid=')[-1].split('&')[0]
                ajax_page_url = f'{org_url}/ajax_page/get_content_area.cfm?mid={mid}&i_list_number=250&i_page=1&i_sory_by=1'
                tgt = pd.read_html(ajax_page_url, encoding='utf-8')[0]
                tgt['圖書館'], tgt['連結'] = org, url
                tgts.append(tgt)
                table = pd.concat(tgts, axis=0, ignore_index=True)
                table = organize_columns(table)
                return table  # 完成抓取 table
            except:  # 沒有搜尋結果，也沒有進入＂詳細書目頁面＂
                print(f'《{ISBN}》查無此書')
                return  # 什麼都不做，退出此 function

        # 抓取多個＂詳細書目＂的網址
        anchors = driver.find_elements_by_link_text('詳細書目')
        urls = []
        for anchor in anchors:
            urls.append(anchor.get_attribute('href'))

        # 抓取方式：找出 mid 後，進入 ajax pag 抓取 DataFrame
        org_url = org_url.replace('/search.cfm', '')
        tgts = []
        for url in urls:
            mid = url.split('mid=')[-1].split('&')[0]  # 抓取 mid
            ajax_page_url = f'{org_url}/ajax_page/get_content_area.cfm?mid={mid}&i_list_number=250&i_page=1&i_sory_by=1'
            tgt = pd.read_html(ajax_page_url, encoding='utf-8')[0]
            tgt['圖書館'], tgt['連結'] = org, url
            tgts.append(tgt)
        table = organize_columns(table)
        return table  # 完成抓取 table
    except:
        print(f'《{ISBN}》在「{url}」無法爬取')

### 函式測試

In [ ]:
if __name__ == '__main__':
    driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)
    table = webpac_ajax_page_crawler(
        org='新北市立圖書館',
        org_url='https://webpac.tphcc.gov.tw/webpac/search.cfm',
        ISBN='9789869109321'
    )
    table = webpac_ajax_page_crawler(
        org='高雄市立空中大學',
        org_url='https://webpac.ouk.edu.tw/webpac/search.cfm',
        ISBN='9789869109321'
    )
    table = webpac_ajax_page_crawler(
        org='國立屏東大學',
        org_url='https://webpac.nptu.edu.tw/webpac/search.cfm',
        ISBN='9789869109321'
    )

## 基隆市公共圖書館(org, org_url, ISBN)
- 『函式完成度』：中

### 函式說明
- 『運作的原理』：使用 Selenium
- 『適用的機構』：[基隆市公共圖書館](https://webpac.klccab.gov.tw/webpac/search.cfm)
- 『能處理狀況』：一筆、無
- 『下一步優化』：
    - 網站載入過慢，且 wait 方式不適用於此，只能使用大量的 time.sleep()

### 函式本體

In [ ]:
def 基隆市公共圖書館(org, org_url, ISBN):
    try:
        # 進入＂搜尋主頁＂
        driver.get(org_url)
        # 等待點擊＂進階查詢＂按鈕，接著點擊
        WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.LINK_TEXT, '進階檢索'))).click()
        time.sleep(2)  # JavaScript 動畫，強制等待
        # 等待定位＂下拉式選單＂，選擇以 ISBN 方式搜尋
        search_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'as_type_1')))
        select = Select(search_field)
        select.select_by_value('i')
        # 定位＂搜尋欄＂，輸入 ISBN
        search_input = driver.find_element_by_id('as_keyword_1')
        search_input.send_keys(ISBN)
        search_input.send_keys(Keys.ENTER)

        time.sleep(8)  # 基隆的系統太詭異了，強制等待
        soup = BeautifulSoup(driver.page_source, "html.parser")
        results = len(soup.find_all("div", "list_box"))
        if results < 2:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "table.list.list_border")))
            time.sleep(2)
            table = pd.read_html(driver.page_source)[0]
        else:
            table = []
            for li in soup.find_all("div", "list_box"):
                url_temp = "https://webpac.klccab.gov.tw/webpac/" + li.find(
                    "a", "btn")["href"]
                driver.get(url_temp)
                wait.until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "table.list.list_border")))
                time.sleep(2)
                table.append(
                    pd.read_html(driver.page_source, encoding="utf-8")[0])
            table = pd.concat(table, axis=0, ignore_index=True)
        table['圖書館'], table['連結'] = org, driver.current_url
        table = organize_columns(table)
        return table
    except:
        print(f'《{ISBN}》在「{url}」無法爬取')

### 函式測試

In [ ]:
if __name__ == '__main__':
    driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)
    table = 基隆市公共圖書館(
        org='基隆市公共圖書館',
        org_url='https://webpac.klccab.gov.tw/webpac/search.cfm',
        ISBN='9789869109321'
    )

## 臺北市立圖書館(org, org_url, ISBN)
- 『函式完成度』：極高

### 函式說明
- 『運作的原理』：輸入 ISBN 搜索後，直接進入＂詳細書目＂，並且該頁面有全部的藏書狀況，只要爬一個表格即可。
- 『適用的機構』：[臺北市立圖書館](https://book.tpml.edu.tw/webpac/webpacIndex.jsp)
- 『能處理狀況』：找不到、一筆
- 『下一步優化』：暫無問題

### 函式本體

In [ ]:
def 臺北市立圖書館(org, org_url, ISBN):
    try:
        # 進入＂搜尋主頁＂
        driver.get(org_url)
        # 等待定位＂下拉式選單＂，選擇以 ISBN 方式搜尋
        search_field = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.NAME, 'search_field')))
        select = Select(search_field)
        select.select_by_value('ISBN')
        # 定位＂搜尋欄＂，輸入 ISBN
        search_input = driver.find_element_by_name('search_input')
        search_input.send_keys(ISBN)
        search_input.submit()

        # 等待＜表格＞出現
        WebDriverWait(driver, 30).until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, 'table.order')))

        # 取得當前網頁的 html 進行解析，以取得 DataFrame
        tgt = pd.read_html(driver.page_source, encoding="utf-8")
        table = tgt[-3]
        table['圖書館'], table['連結'] = org, driver.current_url
        table = organize_columns(table)
        return table
    except:
        print(f'《{ISBN}》在「{org_url}」無法爬取')

### 函式測試

In [ ]:
if __name__ == '__main__':
    driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)
    table = 臺北市立圖書館(
        org='臺北市立圖書館',
        org_url='https://book.tpml.edu.tw/webpac/webpacIndex.jsp',
        ISBN='9789869109321'
    )

## 國家圖書館(org, org_url, ISBN)
- 『函式完成度』：極高

### 函式說明
- 『運作的原理』：使用 Selenium
- 『適用的機構』：[國家圖書館](https://aleweb.ncl.edu.tw/F)
- 『能處理狀況』：找不到、一筆
- 『下一步優化』：
    - 目前尚未遇到多筆情況
    - 不知道可以和什麼機構的系統合併在一起？

### 函式本體

In [ ]:
def 國家圖書館(org, org_url, ISBN):
    try:
        driver.get(org_url)
        select_ISBN_strategy('find_code', 'ISBN')
        search_ISBN(ISBN, 'request')
        
        # 點擊＂書在哪裡(請點選)＂，進入＂詳細書目＂
        tgt_url = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, '書在哪裡(請點選)'))).get_attribute('href')
        driver.get(tgt_url)
        # 定位＂表格＂，爬取內容
        accurately_find_table_and_read_it('')
        
        table = pd.read_html(driver.page_source)[-2]
        table['圖書館'], table['連結'] = org, tgt_url
        table = organize_columns(table)
        return table
    except:
        print(f'《{ISBN}》在「{org_url}」無法爬取')

### 函式測試

In [ ]:
if __name__ == '__main__':
    driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)
    table = 國家圖書館(
        org='國家圖書館',
        org_url='https://aleweb.ncl.edu.tw/F',
        ISBN='9789869109321'
    )

## 彰化縣公共圖書館(org, org_url, ISBN) 進行中
- 『函式完成度』：高

### 函式說明
- 『運作的原理』：待輸入
- 『適用的機構』：彰化縣圖書館
- 『能處理狀況』：一筆、無、多筆、（翻頁
- 『下一步優化』：
    - [翻頁](https://library.toread.bocach.gov.tw/toread/opac/bibliographic_view?NewBookMode=false&id=341724&mps=10&q=986729193X+OR+9789867291936&start=0&view=CONTENT)
    - 待輸入

### 函式本體

In [ ]:
def 彰化縣公共圖書館(org, org_url, ISBN):
    table = []
    tgt_urls = []
    
    driver.get(org_url)
    search_ISBN(ISBN, 'q')
    
    if not wait_for_element_present('div#results'):
        print(f'在{org}裡，沒有《{ISBN}》')
        return
    
    # 有 div#results，接著分析 html，找出所有的＂詳細書目＂的網址
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    ul = soup.select('div#results > ul')[0]
    for li in ul.select('li.is_img'):
        tgt_urls.append(org_url + li.a['href'])
    # for 迴圈，進入所有的＂詳細書目＂的網址
    for tgt_url in tgt_urls:
        driver.get(tgt_url)
        tgt = accurately_find_table_and_read_it('table.gridTable')
        
        # 以下兩行，是＂彰化縣公共圖書館＂有多餘的 row，須要特別篩選調 NaN
        filtered_tgt = tgt.dropna(subset=['典藏地名稱'])
        filtered_tgt.reset_index(drop=True, inplace=True)
        
        table.append(filtered_tgt)
    table = organize_columns(table)
    return table

### 函式測試

In [ ]:
if __name__ == '__main__':
    driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)
    table = 彰化縣公共圖書館(
        org='彰化縣公共圖書館',
        org_url='https://library.toread.bocach.gov.tw',
        ISBN='986729193X'
    )

In [ ]:
table

## 待輸入(org, org_url, ISBN)
- 『函式完成度』：待輸入

### 函式說明
- 『運作的原理』：待輸入
- 『適用的機構』：待輸入
- 『能處理狀況』：待輸入
- 『下一步優化』：
    - 待輸入
    - 待輸入

### 函式本體

In [ ]:
def web2_crawler(org, url_front, ISBN, url_behind, driver):
    url = url_front + ISBN + url_behind
    try:
        driver.get(url)
        title = driver.find_element_by_xpath('/html/body/div/div[1]/div[2]/div/div/div[2]/div[3]/div[1]/div[3]/div/ul/li/div/div[2]/h3/a').click()
        
        df_web2 = accurately_find_table_and_read_it('div#LocalHolding > table')
        df_web2 = organize_columns(df_web2)
        return df_web2
    except:
        print(f"「{url}」無法爬取！")

### 函式測試

In [ ]:
if __name__ == '__main__':
    driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)
    table = web2_crawler(
        org='國立勤益科技大學',
        url_front='http://140.128.95.172/webpac/search/?q=',
        url_behind='&field=isn&op=AND&type=&department=&college=&dep=&active=1',
        ISBN='9789869109321'
    )
table

In [ ]:
soup.find('div', {'id': 'LocalHolding'})

In [ ]:
soup2 = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
soup2.find('div', {'id': 'LocalHolding'})

In [ ]:
type(df_web2)

In [ ]:
df_web2['圖書館'], df_web2['連結'] = org, driver.current_url
df_web2 = organize_columns(df_web2)

## 連江縣公共圖書館(org, org_url, ISBN)
- 最後更新：7/27
- 『函式完成度』：極高

### 函式說明
- 『運作的原理』：待輸入
- 『適用的機構』：[連江縣公共圖書館](http://210.63.206.76/Webpac2/msearch.dll/)、[開南大學](http://www.lib.knu.edu.tw/Webpac2/msearch.dll/)（無法？
- 『能處理狀況』：一筆、無
- 『下一步優化』：暫無找到多筆

### 函式本體

In [ ]:
def 連江縣公共圖書館(org, org_url, ISBN):
    table = []
    
    driver.get(org_url)
    search_ISBN(ISBN, 'ISBN')

    if wait_for_element_present('重新查詢', by=By.LINK_TEXT):
        print(f'在「{org}」找不到「{ISBN}」')
        return
    
    tgt = accurately_find_table_and_read_it('table', -2)
    tgt['圖書館'], tgt['連結'] = org, driver.current_url
    table.append(tgt)
    
    table = organize_columns(table)
    return table

### 函式測試

In [ ]:
if __name__ == '__main__':
    driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)
    table = 連江縣公共圖書館(
        org='開南大學',
        org_url='http://www.lib.knu.edu.tw/Webpac2/msearch.dll/',
        ISBN='9789573317241'
    )


## 樹德科技大學(org, org_url, ISBN) 進行中
- 『函式完成度』：待輸入

### 函式說明
- 『運作的原理』：待輸入
- 『適用的機構』：待輸入
- 『能處理狀況』：待輸入
- 『下一步優化』：
    - 待輸入
    - 待輸入

### 函式本體

In [11]:
driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)

In [12]:
ISBN='9789869109321'

In [25]:
def 樹德科技大學(org, org_url, ISBN):
    table = []
    driver.get('https://webpac.stu.edu.tw/webopac/')
    iframe = driver.find_element(By.NAME, 'default')
    driver.switch_to.frame(iframe)
    select_ISBN_strategy('ctl00$ContentPlaceHolder1$ListBox1', 'Info000076')
    search_ISBN(ISBN, 'ctl00$ContentPlaceHolder1$TextBox1')
    driver.switch_to.default_content()
    iframe = driver.find_element(By.NAME, 'default')
    driver.switch_to.frame(iframe)
    driver.find_elements(By.CSS_SELECTOR, '#ctl00_ContentPlaceHolder1_dg_ctl02_lbtgcd2')[0].click()
    tgt = accurately_find_table_and_read_it('#ctl00_ContentPlaceHolder1_dg')
    tgt['圖書館'], tgt['連結'] = '樹德科技大學', driver.current_url
    table.append(tgt)
    table = organize_columns(table)
    return table

In [28]:
樹德科技大學('樹德科技大學', 'https://webpac.stu.edu.tw/webopac/', '986729193XX')

,圖書館,館藏地,索書號,館藏狀態,連結
0,樹德科技大學,圖書館 B1中文書庫區(0/1),550 4022 2010,C344801/仍在館內,https://webpac.stu.edu.tw/webopac/
1,樹德科技大學,圖書館 B1中文書庫區(0/1),550 4022,C276957/仍在館內,https://webpac.stu.edu.tw/webopac/


### 函式測試

In [ ]:
if __name__ == '__main__':
    driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)
    table = 待輸入(
        org='待輸入',
        org_url='待輸入',
        ISBN='9789869109321'
    )

## 國立臺北護理健康大學(org, org_url, ISBN)
- 『函式完成度』：高

### 函式說明
- 『運作的原理』：待輸入
- 『適用的機構』：[國立臺北護理健康大學](http://140.131.94.8/uhtbin/webcat)、[大同大學](http://140.129.23.14/uhtbin/webcat)、[國立體育大學](http://192.83.181.243/uhtbin/webcat)
- 『能處理狀況』：待輸入
- 『下一步優化』：
    - 待輸入
    - 待輸入

### 函式本體

In [125]:
def 國立臺北護理健康大學(org, org_url, ISBN):
    try:
        driver.get(org_url)
        try:
            select_ISBN_strategy('srchfield1', 'GENERAL^SUBJECT^GENERAL^^所有欄位')
        except:
            select_ISBN_strategy('srchfield1', '020^SUBJECT^SERIES^Title Processing^ISBN')
        search_ISBN(ISBN, 'searchdata1')
        tgt = accurately_find_table_and_read_it('table', 0)
        tgt.drop([0, 1, 2], inplace=True)
        tgt.drop([1, 2, 4], axis='columns', inplace=True)
        tgt.rename(columns={0: '索書號', 3: '館藏狀態'}, inplace=True)
        tgt['圖書館'], tgt['連結'], tgt['館藏地'] = org, driver.current_url, tgt['館藏狀態']
        tgt = organize_columns(tgt)
    except:
        print(f'在「{org}」找不到「{ISBN}」')
        return
    else:
        return tgt

### 函式測試

In [126]:
if __name__ == '__main__':
    driver = webdriver.Chrome(options=my_options, desired_capabilities=my_capabilities)
    table = 國立體育大學(
        org='國立體育大學',
        org_url='http://192.83.181.243/uhtbin/webcat',
        ISBN='9789574672028'
    )
table

,圖書館,館藏地,索書號,館藏狀態,連結
0,國立臺北護理健康大學,4F中日文圖書區,563.081 1838 V.4,4F中日文圖書區,http://192.83.181.243/uhtbin/cgisirsi/?ps=1FHa...
1,國立臺北護理健康大學,4F中日文圖書區,563.081 1838 V.4,4F中日文圖書區,http://192.83.181.243/uhtbin/cgisirsi/?ps=1FHa...
2,國立臺北護理健康大學,4F中日文圖書區,563.081 1838 V.4,4F中日文圖書區,http://192.83.181.243/uhtbin/cgisirsi/?ps=1FHa...
3,國立臺北護理健康大學,4F中日文圖書區,563.081 1838 V.4,4F中日文圖書區,http://192.83.181.243/uhtbin/cgisirsi/?ps=1FHa...
4,國立臺北護理健康大學,外借中 (不流通),563.081 1838 V.4,外借中 (不流通),http://192.83.181.243/uhtbin/cgisirsi/?ps=1FHa...
